### 1. 問題の分析

#### 競技プログラミング視点

* まず長さチェック：
  `s1.length + s2.length !== s3.length` なら **絶対に不可能** なので即 `false`。
* 典型的な **2 次元 DP** 問題：

  * `dp[i][j]`: `s1` の先頭 `i` 文字と `s2` の先頭 `j` 文字を interleave して `s3` の先頭 `i + j` 文字を作れるかどうか。
  * 遷移：

    * `s1` から 1 文字使う場合
      `dp[i][j] |= dp[i-1][j] && s1[i-1] === s3[i+j-1]`
    * `s2` から 1 文字使う場合
      `dp[i][j] |= dp[i][j-1] && s2[j-1] === s3[i+j-1]`
* 制約は `|s1|, |s2| ≤ 100` なので
  時間 `O(|s1| * |s2|) ≤ 10^4` は十分高速。
* Follow up にある通り、2D DP を **1 次元に圧縮**することで
  追加メモリを `O(|s2|)` に抑えられる。

#### 業務開発視点

* 再帰 DFS だとスタックオーバーフローや経路爆発のリスクがあるので、
  **ループベースの DP** が安全・予測可能。
* DP 配列はローカル変数のみを使用し、関数は **完全に純粋関数**（副作用なし）。
* TypeScript の型で引数はすべて `string` に限定されるので、
  実行時エラーの可能性を最小化できる。
* LeetCode 環境では異常入力は来ない前提だが、汎用ライブラリとして考えるなら
  軽い型チェックを入れて `TypeError` を投げる設計もあり。

#### TypeScript 特有の考慮点

* 入力はすべて `string` なので、ジェネリクスは過剰になりがち。
  → **シンプルなシグネチャ** `isInterleave(s1: string, s2: string, s3: string): boolean`
* `const`・`readonly` を徹底して、
  DP 配列以外はイミュータブルに扱い、意図しない再代入を防止。
* `strictNullChecks` 前提で `null` / `undefined` を扱わない設計にすることで
  余計な分岐を減らし、読みやすさも担保。

---

### 2. アルゴリズムアプローチ比較

| アプローチ                  | 時間計算量             | 空間計算量              | TS実装コスト | 型安全性 | 可読性 | 備考                  |
| ---------------------- | ----------------- | ------------------ | ------- | ---- | --- | ------------------- |
| 方法A: 単純DFSバックトラック      | `O(2^(n1+n2))` 最悪 | `O(n1+n2)`（再帰スタック） | 低       | 高    | 中   | すぐ TLE / 爆発。競プロではNG |
| 方法B: 2D DP（`dp[i][j]`） | `O(n1 * n2)`      | `O(n1 * n2)`       | 中       | 高    | 高   | 実装素直、理解しやすい         |
| 方法C: 1D DP（列方向圧縮）      | `O(n1 * n2)`      | `O(min(n1, n2))`   | 中       | 高    | 中〜高 | Follow up 要件 `O(    \| s2 \| )` を満たす |

---

### 3. 選択したアルゴリズムと理由

* **選択したアプローチ**: 方法C — 1D DP（空間最適化版）

#### 理由

* 時間計算量は 2D DP と同じ `O(n1 * n2)` だが、
  メモリ使用量を `O(min(n1, n2))` まで削減できる。
* 実装は 2D DP をそのまま圧縮した形なので、
  コメントを丁寧に書けば可読性も十分確保可能。
* TypeScript では `boolean[]` の一次元配列のみを扱えば良く、
  型も単純で扱いやすい。

#### TypeScript 特有の最適化ポイント

* `s1`, `s2` の長さを比較し、短い方を DP の列方向にすることで
  追加メモリを常により小さい方に抑える。
* ループ内のインデックスを `const` で束縛し、
  誤更新や off-by-one バグを防ぎやすくする。

---

### 4. 実装コード（LeetCode 用 TypeScript）

LeetCode の TypeScript 回答フォーマットに合わせて、
**グローバル関数のみ** を定義します（`export` なし）。

```typescript
/**
 * Interleaving String 判定
 * s3 が s1 と s2 の interleaving で構成できるかどうかを判定する。
 *
 * @param s1 - 1つ目の文字列
 * @param s2 - 2つ目の文字列
 * @param s3 - 判定対象の文字列
 * @returns s3 が s1, s2 の interleaving なら true、それ以外は false
 *
 * @complexity Time: O(n1 * n2), Space: O(min(n1, n2))
 */
function isInterleave(s1: string, s2: string, s3: string): boolean {
    // --- 型レベルでは string だが、汎用性を考慮した軽い実行時ガード ---
    if (typeof s1 !== "string" || typeof s2 !== "string" || typeof s3 !== "string") {
        throw new TypeError("All inputs must be strings");
    }

    const len1 = s1.length;
    const len2 = s2.length;
    const len3 = s3.length;

    // 長さが合わなければ interleaving は不可能
    if (len1 + len2 !== len3) {
        return false;
    }

    // メモリ最適化: 列方向に使う方（後で DP 配列の長さになる）を常に短い文字列にする
    // a: 行方向、b: 列方向
    let a = s1;
    let b = s2;
    let lenA = len1;
    let lenB = len2;

    if (lenB > lenA) {
        // swap
        a = s2;
        b = s1;
        lenA = len2;
        lenB = len1;
    }

    // dp[j]: a の先頭 i 文字と b の先頭 j 文字で s3 の先頭 i + j 文字を構成できるか
    const dp: boolean[] = new Array<boolean>(lenB + 1).fill(false);
    dp[0] = true;

    // i = 0 行（a を 0 文字使う場合）の初期化
    for (let j = 1; j <= lenB; j++) {
        dp[j] = dp[j - 1] && b[j - 1] === s3[j - 1];
    }

    // i = 1..lenA についてループ
    for (let i = 1; i <= lenA; i++) {
        // j = 0 列（b を 0 文字使う場合）の更新
        dp[0] = dp[0] && a[i - 1] === s3[i - 1];

        for (let j = 1; j <= lenB; j++) {
            const k = i + j - 1; // s3 のインデックス

            const fromA = dp[j] && a[i - 1] === s3[k];      // 上から来る（a の文字を使う）
            const fromB = dp[j - 1] && b[j - 1] === s3[k];  // 左から来る（b の文字を使う）

            dp[j] = fromA || fromB;
        }
    }

    return dp[lenB];
}

Analyze Complexity
Runtime 42 ms
Beats 98.45%
Memory 56.79 MB
Beats 82.17%

```

* 2D DP を 1D に圧縮しているだけなので、
  理解しづらい場合はまず 2D 版を紙に書いてみると直感的に追えるはずです。
* `a` / `b` のスワップは純粋にメモリ削減のためで、
  **結果には一切影響しません**（interleaving は左右対称の性質を持つため）。

結論から言うと、**アルゴリズムとしてはほぼ最適**なので、これ以上の改善は「理論的な高速化」ではなく、**定数倍の微調整**レベルになります 👍

今の成績：

* Runtime: 42ms（**98.45% を上回る**）
* Memory: 56.79MB（**82.17% を上回る**）

この時点で、実戦レベルでは十分すぎる性能です。
そのうえで、「やるとしたらここかな」という改善ポイントをいくつか挙げます。

---

## どこに改善の余地があるか？

### 1. 実行時の型チェックを削る（LeetCode 前提なら不要）

前回の実装ではこんなチェックを入れていました：

```ts
if (typeof s1 !== "string" || typeof s2 !== "string" || typeof s3 !== "string") {
    throw new TypeError("All inputs must be strings");
}
```

LeetCode では **必ず string が渡ってくる前提**なので、
このガードは「安全性」には寄与せず、純粋にオーバーヘッドです。

👉 **競プロ環境では削って OK**
（業務コードで再利用したいなら残してもよいですが、LeetCode 用なら外してしまう方がシンプル＆わずかに高速）

---

### 2. 「短い方を列にする」ための swap をやめる

前回はメモリをさらに削るために、

```ts
let a = s1;
let b = s2;
// ...
if (lenB > lenA) {
    // swap a, b
}
```

のように、**短い方を列方向にする最適化**を入れていました。

ただしこの問題の制約はどちらも最大 100 文字なので、

* 2D でも最大 `101 x 101` ≒ 1 万セル
* 1D でも最大 101 要素

と、どちらにしても **誤差レベル**です。
swap の分岐が入るぶん、コードが少し複雑になり、分岐コストも増えます。

👉 メモリが十分小さいので、**常に `s2` を列（dp の長さ）に固定**してしまったほうが、

* 実装がスッキリ
* 分岐も減って、微妙に速くなる可能性

があります。

---

### 3. 2D DP ではなく 1D DP を維持（ここはもう最適）

* 時間計算量：`O(|s1| * |s2|)` は **理論的にほぼベスト**
* 1D DP にしているので、追加メモリ `O(|s2|)` も **follow up 要件をクリア**

ここはすでにベストチョイスなので、いじる必要はありません。

---

## 微調整版 TypeScript 実装（LeetCode 最適化寄り）

LeetCode 用に「余計な型チェック・swap を削った」バージョンです。
アルゴリズムは同じ 1D DP ですが、コードを一段シンプルにしています。

```typescript
function isInterleave(s1: string, s2: string, s3: string): boolean {
    const n1 = s1.length;
    const n2 = s2.length;
    const n3 = s3.length;

    // 長さが合わなければ絶対不可能
    if (n1 + n2 !== n3) {
        return false;
    }

    // dp[j]: s1 の先頭 i 文字 + s2 の先頭 j 文字で s3 の先頭 i + j 文字を作れるか
    const dp: boolean[] = new Array<boolean>(n2 + 1).fill(false);

    // i = 0（s1 を 0 文字使う）行の初期化
    dp[0] = true;
    for (let j = 1; j <= n2; j++) {
        dp[j] = dp[j - 1] && s2[j - 1] === s3[j - 1];
    }

    for (let i = 1; i <= n1; i++) {
        // j = 0 列（s2 を 0 文字使う）を更新
        dp[0] = dp[0] && s1[i - 1] === s3[i - 1];

        for (let j = 1; j <= n2; j++) {
            const k = i + j - 1; // s3 のインデックス

            // 上から来る：s1 の文字を使う
            const fromS1 = dp[j] && s1[i - 1] === s3[k];
            // 左から来る：s2 の文字を使う
            const fromS2 = dp[j - 1] && s2[j - 1] === s3[k];

            dp[j] = fromS1 || fromS2;
        }
    }

    return dp[n2];
}

Analyze Complexity
Runtime 54 ms
Beats 60.46%
Memory 55.56 MB
Beats 96.90%

```

### この修正版で変わるポイント

* **削ったもの**

  * 実行時 `typeof` チェック
  * `s1` / `s2` の長さによる swap 分岐
* **維持しているもの**

  * 1D DP による `O(n1 * n2)` 時間・`O(n2)` 空間
  * ロジックの安全性・可読性

---

## まとめ

* 今の 42ms / 56.79MB というスコアは、**アルゴリズムレベルでは十分最適**です。
* これ以上は

  * 実行時型チェックを削る
  * swap 最適化を削ってロジックをシンプルにする
    といった **定数倍の微チューニング**になります。
* LeetCode の runtime は環境によるブレも大きいので、
  体感では「もうこれで完成」と思って問題ないレベルです 💪
